# Constrained Optimization of the Full Transmon Model

In [1]:
import os
from os.path import join
import sys
import numpy as np
import matplotlib.pylab as plt

$\newcommand{\Op}[1]{\boldsymbol{\mathsf{\hat{#1}}}}$

## Introduction

We consider the full Transmon Hamiltonian
\begin{equation}
  \Op{H} = \sum_{q=1,2} \left[ \omega_q \Op{b}^{\dagger}_q \Op{b}_q
    + \frac{\alpha_q}{2} \Op{b}^{\dagger}_q\Op{b}^{\dagger}_q\Op{b}_q\Op{b}_q
    + g_q \left(\Op{b}^{\dagger}_q \Op{a} + \Op{b}_q \Op{a}^{\dagger} \right)
    \right]
    + \omega_c \Op{a}^{\dagger}\Op{a}
    + \epsilon^{*}(t) \Op{a} + \epsilon(t) \Op{a}^{\dagger}
\end{equation}
Note that unlike in the optimizations done for KITP in 2013, we do *not* have a static qubit-qubit coupling, outside the effective coupling that is mediated by the cavity

Goals:

1.  Scan the parameter regime systematically for the point that allows the
    easiest implementation of both single- and two-qubit gates. Identify
    known gate mechanisms on map. The scan shows:
    *   Which parameters yield the fastest gates
    *   Which local equivalence classes different qubit parameters lead to
    *   Whether there are "good" regions in addition to the regions at which known gate mechanism have been implemented (determined by resonance conditions)

2.  For select points, determine the quantum speed limit for high fidelity
    gates: What are the fastest possible quantum gates with all-microwave control of transmon qubits 
3.  Include robustness with respect to dissipation, and fluctuations in the
    qubit parameters

Plan:

*   Prerequisite: Method testing
    *   Test optimization with state-dependent and frequency constraints
    *   Test optimization with perfect entangler
    *   Test optimization for single-qubit gate
    *   Examine the RIP method for depopulating the cavity. If generally applicable, incorporate into all guess pulses below
    
*   Scan parameter space.

    We choose $\omega_1$, $\alpha_1 \neq \alpha_2$, $g_1 = g_2$ constant. We vary $\omega_2$ and $\omega_c$ (in the RWA) on a grid. At each grid point (set of paremters), we perform the following steps to evaluate the parameters as a candidate:
    
    *   Set $T=200$ ns (possibly also $T = 500$ ns and $T = 100$ ns
    *   Determine a good guess pulse for the optimization towards a perfect entangler, and for the optimization towards a single-qubit gate
    *   Perform a Krotov or LBFGS optimization, for PE and for single qubit gates (either LI-optimization for identity, or two optimizations $\sigma_x^{(1)}$ and $\sigma_x^{(2)}$.
    
    The critical step is the determination of a good guess pulse. A possiblity is to generate simple guess pulses based on some randomized parameters, and to perform a few simplex steps in these parameters.
    
*  Pick several points in the parameter space that yielded the best success. Determin the QSL by optimizing at increasingly smaller values of $T$.
    

## Environment Setup

In [2]:
%install_ext http://raw.github.com/goerz/version_information/master/version_information.py
%reload_ext version_information
% version_information numpy, matplotlib, pandas, QDYN, QDYNTransmonLib, "cd $PREFIX/src/qdyn && git log",  "cd $PREFIX/src/transmon_oct && git log"

Installed version_information.py. To use it, type:
  %load_ext version_information


Software,Version
Python,2.7.10 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython,3.1.0
OS,Linux 3.13.0 45 generic x86_64 with debian jessie sid
numpy,1.9.2
matplotlib,1.4.3
pandas,0.16.1
QDYN,0.1.0
QDYNTransmonLib,1.0.2
cd $PREFIX/src/qdyn && git log,commit f1e3bb93036a3c00abde0151279eebcf6cc605fb
cd $PREFIX/src/transmon_oct && git log,commit 72d26c92b2f6cb55849c4db8b1d42155d05ddb16


In [3]:
PREFIX = join(os.getcwd(), 'venv')
assert os.path.samefile(join(PREFIX, 'bin'), os.environ['PATH'].split(os.pathsep)[0])

## Prerequisites

*   [Does state-dependent constraint work?](Prereq_Nonherm.ipynb) Yes, but slow convergence
*   [Can we ensure a small nq,nc? What sampling rate is sufficient?](Prereq_Cutoff.ipynb) One extra level is sufficient. 100 samples per oscillation is sufficient if we don't claim high precision

## Qubit Paramters

*   $\omega_1/2\pi =$ 6.0 GHz
*   $\omega_2/2\pi =$ 6.25, 6.5, 6.75, 7.0, 7.25, 7.5, 7.75, 8.0, 8.5, 9.0 GHz
*   $\omega_c/2\pi =$ 6.6, 7.1, 7.6, 8.1, 8.6, 9.1, 9.6, 10.1, 10.6, 11.1 GHz (enforce $\omega_c > \omega_2$)  
*   $\alpha_1/2\pi =$ 290 MHz
*   $\alpha_2/2\pi =$ 310 MHz
*   $g/2\pi =$ 70 MHz
*   $T = $ 200 ns

We take into account 4 qubit levels with realistic decay time [Peterer et al, Phys. Rev. Lett. 114, 010501 (2015)], and 5 cavity levels, also with realistic decay [A. W. Cross and J. M. Gambetta, Phys. Rev. A 91 032325 (2015)]. The last level for each qubit and the cavity is "truncated" by setting the decay to "infinity", as an effective state-dependent constraint.

Decay is modelled by a non-Hermitian Hamiltonian (decay leads to loss of norm)

## Stage 1: Random Frequency Search & Amplitude Scan

For each parameters pair ($\omega_2, \omega_c$) we run a `./pre_simplex_scan.py` script, taking the following steps:
*   create runfolders, e.g. `w2_6000MHz_wc_11000MHz/stage1/...`
    
        .../field_free -> determine zeta
        .../1_freq_center <- frequency centered between qubits
        .../1_freq_1..10 <- 10 random frequencies
        .../2_freq_resonant <- resonant with both qubits
        .../2_freq_1..10 <- 10 random frequency pairs
        .../5_freq_1..10 <- 10 random frequency/amplitude choices
        [33 runfolders per parameter set]

*   for each runfolder, scan peak amplitude over values $E_0 =$ 10, 50, 100, 150, 200, 250, 300, 350, 400, 450 MHz   
    [total 10 * 33 propagations per parameter set]

*   select best runs, and create runfolders for stage, e.g. `w2_6000MHz_wc_11000MHz/stage1/...`

        .../PE_1freq_center <- best amplitude for perfect entangler
        .../O_1freq_center <- best amplitude for LEC unity
        .../PE_1freq_random <- best random frequency/amplitude for PE
        .../O_1freq_random
        .../PE_2freq_resonant
        .../O_2freq_resonant
        .../PE_2freq_random
        .../O_2freq_random
        .../PE_5freq_random
        .../O_5freq_random
        [10 runfolders per parameter set]
        

In [11]:
! ./run_stage1.py

/bin/sh: 1: ./run_stage1.py: not found


## Stage 2: Simplex (Pre-)Optimization

For each parameter pair, and each of the 10 runfolders selected for each parameter pair,

*   run simplex search. Figure of merit is either PE, or local equivalence class of the identity (local gates). All    pulses use Blackman shape $B(t)$
    
    parameters to vary are:
    * for single-frequency $\epsilon(t) = E_0 B(t) \cos(\omega_L t)$: $E_0$, $\omega_L$ [2 parameters]
    * for 2 frequencies $\epsilon(t) = B(t)(a_1 \cos(\omega_{L,1} t) + a_2 \cos(\omega_{L,2} t + \phi_0)$: $a_1$, $a_2$, $\omega_{L,1}$, $\omega_{L,2}$, $\phi_0$. [5 parameters]
    * for 5 frequencies $\epsilon(t) = B(t)\sum_{n=1}^{5}(a_n \cos(\omega_{L,n} t) + b_n \sin(\omega_{L,n} t))$: $a_{1..5}$, $b_{1..5}$, $\omega_{L,1..5}$ [15 parameters]

In [ ]:
! ./run_stage2.py

## Stage 3: Gradient-Based Search

* Run Krotov for 50 iterations on all runfolders
* For each parameter set, select the one most promising run
* Continue optimization

## Stage 4: Quantum Speed Limit

For best points resulting from stage 3, lower gate duration from $T = $ 200 ns to determine QSL